# Libraries and Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/CI for PD")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import linalg as la
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm, trange
from numba import jit, njit
try:
    import multiprocess
except:
    import multiprocessing as multiprocess
import warnings
warnings.filterwarnings('ignore')

In [4]:
%load_ext cython

In [5]:
multiprocess.cpu_count()

2

# Reading data

In [6]:
MappingData = pd.read_csv("ratings_mapping.csv")
MappingData.head()

,RatingSymbol,RatingNumber
0,AAA,0
1,AA+,1
2,AA,2
3,AA-,3
4,A+,4


In [7]:
TransitionData = pd.read_csv("issuer_credit_ratings_cleaned_local_currency_lt.csv")
TransitionData.head()

,rating,rating_action_date,obligor_name,rating_outlook
0,B+,2014-09-15,1011778 B.C. Unlimited Liability Company,Stable
1,B+,2015-06-08,1011778 B.C. Unlimited Liability Company,Stable
2,B+,2016-06-29,1011778 B.C. Unlimited Liability Company,Stable
3,B+,2017-02-14,1011778 B.C. Unlimited Liability Company,Stable
4,B+,2017-02-24,1011778 B.C. Unlimited Liability Company,Stable


In [8]:
TransitionData = TransitionData[["obligor_name", "rating_action_date", "rating"]]
TransitionData.columns = ["Id", "Date", "RatingSymbol"]
TransitionData = TransitionData.sort_values(by=["Id", "Date"])
le = LabelEncoder()
TransitionData.Id = le.fit_transform(TransitionData.Id)

In [9]:
TransitionData.Date = pd.to_datetime(TransitionData.Date)

In [10]:
TransitionData = pd.merge(TransitionData, MappingData, on='RatingSymbol', how='left')
TransitionData.head()

,Id,Date,RatingSymbol,RatingNumber
0,0,2014-09-15,B+,13
1,0,2015-06-08,B+,13
2,0,2016-06-29,B+,13
3,0,2017-02-14,B+,13
4,0,2017-02-24,B+,13


In [11]:
pij = pd.read_csv("pij.csv")
pij=pij.values

In [12]:
TransDenResults = np.array([  138.,    87.,   366.,   865.,  1430.,  2590.,  4145.,  4858.,
        6055.,  5068.,  3406.,  3990.,  4614.,  6054., 10953.,  4865.,
        1236.,   496.,   247.,   145.,     0.])


TransDenLambdaResults = np.array([  133.60547945,    87.        ,   354.24383562,   841.88493151,
        1402.48767123,  2598.98356164,  4132.31780822,  4852.58082192,
        6055.08767123,  4985.28493151,  3408.77808219,  3919.45205479,
        4504.49041096,  5755.66849315, 10239.53972603,  4803.07123288,
        1323.03561644,   502.45479452,   250.41369863,   150.89589041,
           0.        ])

In [13]:
RatingMgrationCohort = np.zeros([21, 22])
RatingMgrationHazard = np.zeros([21, 22])
for i in range(21):
    for j in range(22):
        RatingMgrationCohort[i,j] = pij[i,j] / TransDenResults[i]
        RatingMgrationHazard[i,j] = pij[i,j] / TransDenLambdaResults[i]

#Default Category
RatingMgrationHazard[20,20] = 1.0 # Absorbing
RatingMgrationHazard[20,:] = 0.0 # All the others = 0

### Hazard

In [14]:
def mask_first(x):
  # https://stackoverflow.com/questions/31226142/python-pandas-delete-the-first-row-by-group
  result = np.ones_like(x)
  result[0] = 0
  return result

In [15]:
# def compute_nijk_for_obligor(t, year, yend, TransMatrixResults,
#                              TransDenResults, TransDenLambdaResults, year_df, i):
#   t.set_description(f"Processing Year {year[0]} and ID borrower No: {i}")
#   t.refresh() # to show immediately the update
#   MyDati = year_df[year_df['Id'] == i]
#   # print ("processing ID borrower No: ", i)
#   if (len(MyDati) == 1 and MyDati.iloc[0].RatingSymbol == 'NR'):
#       # when this is true the data is not processed as it contains only one record, equal to state NR
#       print("Only one NR Rating event on Borrower No: ", i)

#   else:
#       trans = TransitionClass(MyDati, yend)
#       trans.Cohort()
#       trans.CohortTransitionMatrix()
#       trans.HazardModel()

#       TransMatrixResults    = trans.TransMatrix + TransMatrixResults
#       TransDenResults       = trans.TransDen    + TransDenResults
#       TransDenLambdaResults = trans.TransDenLambda + TransDenLambdaResults
#   return TransMatrixResults, TransDenResults, TransDenLambdaResults

In [16]:
# https://github.com/dpicone1/Estimating_Credit_Rating_Transition_Matrices_Hazard_Vs_Cohort/blob/master/TransitionClassFile.py
# the default rating symbol is D
# the default rating index  is 20

# the NR rating symbol is NR
# the NR rating index  is 21

# There are 21 initial states AAA, AA+, AA, AA-, A+, A, A-, BBB+, BBB, BBB-, BB+,
#                            BB, BB-, B+, B, B-, CCC+, CCC, CCC-, CC, D
# and 22 final states         AAA, AA+, AA, AA-, A+, A, A-, BBB+, BBB, BBB-, BB+,
#                            BB, BB-, B+, B, B-, CCC+, CCC, CCC-, CC, D, NR
# %%cython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import linalg as la
import datetime as dt
from tqdm import tqdm, trange
from sklearn.preprocessing import LabelEncoder

def mask_first(x):
  # https://stackoverflow.com/questions/31226142/python-pandas-delete-the-first-row-by-group
  result = np.ones_like(x)
  result[0] = 0
  return result



def create_year_data(df, year):
    year_df = df[df.Date.dt.year.isin(year)]
    year_df = year_df.reset_index(drop=True)

    companies_whose_first_rating_is_nr = year_df.groupby("Id")["RatingSymbol"].first()[year_df.groupby("Id")["RatingSymbol"].first()=="NR"].index.tolist()
    year_df_nr_first = year_df[year_df.Id.isin(companies_whose_first_rating_is_nr)]
    mask = year_df_nr_first.groupby(['Id'])['Id'].transform(mask_first).astype(bool)
    year_df_nr_first_cleaned = year_df_nr_first.loc[mask]
    year_df = pd.concat([
        year_df[~year_df.Id.isin(companies_whose_first_rating_is_nr)],
        year_df_nr_first_cleaned
    ])

    # le = LabelEncoder()
    # year_df['Id'] = le.fit_transform(year_df['Id'])

    return year_df


class TransitionClass(object):
    def __init__(self, Dati, yend):

        #Dati is a dataframe containing the info to be processed
        #yend  is final year of the analysis
        self.MyDati = Dati
        self.MyDati = self.MyDati.copy()

        self.MyDati['Year']     = pd.DatetimeIndex(self.MyDati['Date']).year
        self.MyDati['TimeDiff'] = self.MyDati['Date'].diff()

        self.defautYes = False #The general status is that the loan/bond has not defaulted for the cohort model
        self.NRYes     = False #The general status is that the loan/bond is not NR for the cohort model

        self.DefaultedCheck()# check if the data contains a default
        self.NRCheck()       # check if the data contains a NR

        self.size      = len(self.MyDati)
        self.yend      = yend
        self.ybeg      = (self.MyDati.iloc[0].Date).year


    def DefaultedCheck(self):
        # Is there a default?
        self.defaulted     = self.MyDati[self.MyDati['RatingSymbol'] == 'D']
        self.defaultedSize = len(self.defaulted)

        if (self.defaultedSize > 0):

            # remove all the data after the obligor has defaulted
            DefaultDate = self.MyDati[self.MyDati['RatingSymbol'] == 'D']
            date = DefaultDate.iloc[0].Date
            self.MyDati = self.MyDati.drop(self.MyDati[self.MyDati['Date'] > date].index)

            # After removing all addtional data after defaut,
            # we should have only one defaulted record, the first one
            self.defaulted     = self.MyDati[self.MyDati['RatingSymbol'] == 'D']
            self.defaultedSize = len(self.defaulted)
            self.defaultyear = self.defaulted.iloc[0].Year
            self.defaultYes  = True

    def NRCheck(self):
        # Is there a NR without a previous Default?
        self.NRSize = 0
        self.NR     = self.MyDati[self.MyDati['RatingSymbol'] == 'NR']

        if (len(self.NR) > 0 & self.defaultedSize == 1): # Remove the NR record. We do not need it as we stop
            # as soon as the borrower defaults
            self.MyDati = self.MyDati.drop(self.MyDati[self.MyDati['RatingSymbol'] == 'NR'].index)

        if (len(self.NR) > 0 & self.defaultedSize == 0): # only if there is no default, otherwise the default will
            # overseed and stop the algo as soon as there is a default event

            # The obligor might have several NR or after an NR status might come a rating from AAA to C.
            # We remove all the data after the first NR.

            NRDate = self.MyDati[self.MyDati['RatingSymbol'] == 'NR']
            date = NRDate.iloc[0].Date
            self.MyDati = self.MyDati.drop(self.MyDati[self.MyDati['Date'] > date].index)

            # After removing all addtional NR, we should have only one NR record, the oldest one
            self.NR     = self.MyDati[self.MyDati['RatingSymbol'] == 'NR']
            self.NRSize = len(self.NR)
            self.NRyear = self.NR.iloc[0].Year
            self.NRYes  = True

    def Cohort(self):

        self.RatingsBeg = [self.MyDati.iloc[0].RatingSymbol] # containing the rating symbol at the beginning of the period
        self.RatingsEnd = []                                 # containing the rating symbol at the end of the period

        self.RatingsBegIndex = [self.MyDati.iloc[0].RatingNumber]# containing the rating index at the beginning of the period
        self.RatingsEndIndex = []                                # containing the rating index at the end of the period

        if   (self.defaultedSize == 1):
            finalYear = min(self.yend, self.defaultyear)
        elif (self.NRSize > 0):
            finalYear = min(self.yend, self.NRyear)
        else:
            finalYear = self.yend

        for i in range(self.ybeg, finalYear + 1):

            self.MyDatiLoop = self.MyDati[self.MyDati['Year'] == i]

            if(len(self.MyDatiLoop) > 0):
                dateCond         = max(self.MyDatiLoop.Date)
                self.MyDatiLoop2 = self.MyDatiLoop[self.MyDatiLoop['Date'] == dateCond]
                self.RatingsEnd.append(self.MyDatiLoop2.iloc[0].RatingSymbol)
                self.RatingsEndIndex.append(self.MyDatiLoop2.iloc[0].RatingNumber)

            else:
                self.RatingsEnd.append(self.MyDatiLoop2.iloc[0].RatingSymbol)
                self.RatingsEndIndex.append(self.MyDatiLoop2.iloc[0].RatingNumber)

            self.RatingsBeg.append(self.MyDatiLoop2.iloc[0].RatingSymbol)
            self.RatingsBegIndex.append(self.MyDatiLoop2.iloc[0].RatingNumber)

            if(self.RatingsEnd[-1] == 'D' or self.RatingsEnd[-1] == 'NR' ):
                break

        if(self.defautYes):
            self.RatingsEnd[-1]      = 'D'
            self.RatingsEndIndex[-1] = 20

        if(self.NRYes):
            self.RatingsEnd[-1]      = 'NR'
            self.RatingsEndIndex[-1] = 21

        self.RatingsBeg      = self.RatingsBeg[:-1]
        self.RatingsBegIndex = self.RatingsBegIndex[:-1]

    def CohortTransitionMatrix(self):
        self.TransMatrix = np.zeros([21, 22])
        self.TransDen    = np.zeros([21])

        for i in range(len(self.RatingsEndIndex)):
            self.TransMatrix[self.RatingsBegIndex[i], self.RatingsEndIndex[i]] +=1
            self.TransDen[self.RatingsBegIndex[i]] +=1

    def HazardModel(self):

        self.TransDenLambda  = np.zeros([21]) # containing the denominator Hazard Model

        self.DefaultYesHazard = False

        for i in range(1, self.size):

            valore = self.MyDati.iloc[i].TimeDiff.days / 365
            self.TransDenLambda[self.MyDati.iloc[i - 1].RatingNumber] += valore

            if (self.MyDati.iloc[i].RatingSymbol == 'D' or self.MyDati.iloc[i].RatingSymbol == 'NR'):
                self.DefaultYesHazard = True
                break

        #first period
        dbeg = pd.to_datetime(dt.date(self.MyDati.iloc[0].Year, 1, 1))
        valoreBeg = (self.MyDati.iloc[0].Date - dbeg).days/365.0
        self.TransDenLambda[self.MyDati.iloc[0].RatingNumber] += valoreBeg

        # Last Period Analysis
        dfinal = pd.to_datetime(dt.date(self.yend, 12, 31))
        if(self.DefaultYesHazard == False):
            valoreEnd = (dfinal - self.MyDati.iloc[self.size - 1].Date).days/365.0
            self.TransDenLambda[self.MyDati.iloc[self.size - 1].RatingNumber] += valoreEnd

def compute_nijk(df, year):
    year_df = create_year_data(df, year)
    # print(year_df.Date.min(), year_df.Date.max(), year_df.shape)
    size = len(year_df)
    yend = max(year_df.Date).year
    ystart = min(year_df.Date).year
    IDS = year_df.Id.unique()

    # The arrays containing the outputs
    TransMatrixResults       = np.zeros([21, 22])
    TransDenResults          = np.zeros([21])
    TransDenLambdaResults    = np.zeros([21])

    # Run the algo for all obligors
    t = tqdm(IDS,
              desc='Bar desc',
              leave=True
              )
    for i in t:
        t.set_description(f"Processing Year {year[0]} and ID borrower No: {i}")
        t.refresh() # to show immediately the update
        MyDati = year_df[year_df['Id'] == i]
        # print ("processing ID borrower No: ", i)
        if (len(MyDati) == 1 and MyDati.iloc[0].RatingSymbol == 'NR'):
            # when this is true the data is not processed as it contains only one record, equal to state NR
            print("Only one NR Rating event on Borrower No: ", i)

        else:
            trans = TransitionClass(MyDati, yend)
            trans.Cohort()
            trans.CohortTransitionMatrix()
            trans.HazardModel()

            TransMatrixResults    = trans.TransMatrix + TransMatrixResults
            TransDenResults       = trans.TransDen    + TransDenResults
            TransDenLambdaResults = trans.TransDenLambda + TransDenLambdaResults

    # arg_iterable = [(t, year, yend, TransMatrixResults,
    #                          TransDenResults, TransDenLambdaResults, year_df, i)
    # for i in t
    # ]
    # n_jobs=7
    # with multiprocess.Pool(n_jobs) as pool:
    #     result = pool.starmap(compute_nijk_for_obligor, arg_iterable)

    # return result

    return TransMatrixResults, TransDenResults, TransDenLambdaResults

def postprocess_transition_results(result):
  initial_states = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
                     "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
                     "CCC+", "CCC", "CCC-", "CC", "Default"]

  final_states = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
                      "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
                      "CCC+", "CCC", "CCC-", "CC", "Default", "NR"]

  # Unpack the results
  TransMatrixResults, TransDenResults, TransDenLambdaResults = result

  # Convert Transition matrix to data frame
  TransMatrixResults = pd.DataFrame(TransMatrixResults)

  # change the column names
  TransMatrixResults.columns = final_states

  # Change the row indexes
  TransMatrixResults.index = initial_states


  TransDenResults = pd.Series(TransDenResults, index=initial_states)

  TransDenLambdaResults = pd.Series(TransDenLambdaResults, index=initial_states)

  return TransMatrixResults, TransDenResults, TransDenLambdaResults

def compute_nijk_v2(year_df, year, log=False):
    # year_df = create_year_data(df, year)
    # print(year_df.Date.min(), year_df.Date.max(), year_df.shape)
    size = len(year_df)
    yend = max(year_df.Date).year
    ystart = min(year_df.Date).year
    IDS = year_df.Id.unique()

    # The arrays containing the outputs
    TransMatrixResults       = np.zeros([21, 22])
    TransDenResults          = np.zeros([21])
    TransDenLambdaResults    = np.zeros([21])

    # Run the algo for all obligors
    t = IDS
    # t = tqdm(IDS,
    #           # desc='Bar desc',
    #           # leave=True
    #           )
    for i in t:
        # t.set_description(f"Processing Year {year[0]} and ID borrower No: {i}")
        # t.refresh() # to show immediately the update
        MyDati = year_df[year_df['Id'] == i]
        # print ("processing ID borrower No: ", i)
        if (len(MyDati) == 1 and MyDati.iloc[0].RatingSymbol == 'NR' and log==True):
            # when this is true the data is not processed as it contains only one record, equal to state NR
            print("Only one NR Rating event on Borrower No: ", i)

        else:

            try:
              trans = TransitionClass(MyDati, yend)
              trans.Cohort()
              trans.CohortTransitionMatrix()
              trans.HazardModel()


              TransMatrixResults    = trans.TransMatrix + TransMatrixResults
              TransDenResults       = trans.TransDen    + TransDenResults
              TransDenLambdaResults = trans.TransDenLambda + TransDenLambdaResults
            except IndexError:
              size -= 1
              pass

    # arg_iterable = [(t, year, yend, TransMatrixResults,
    #                          TransDenResults, TransDenLambdaResults, year_df, i)
    # for i in t
    # ]
    # n_jobs=7
    # with multiprocess.Pool(n_jobs) as pool:
    #     result = pool.starmap(compute_nijk_for_obligor, arg_iterable)

    # return result

    return TransMatrixResults, TransDenResults, TransDenLambdaResults

def compute_default_probas(result):
  initial_states = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
                     "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
                     "CCC+", "CCC", "CCC-", "CC", "Default"]

  final_states = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
                      "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
                      "CCC+", "CCC", "CCC-", "CC", "Default", "NR"]

  # Unpack result
  TransMatrixResults, TransDenResults, TransDenLambdaResults = result

  RatingMgrationHazard = pd.DataFrame(np.zeros([21, 22]),
                                      index=initial_states,
                                      columns=final_states)
  for i in initial_states:
    for j in final_states:

      RatingMgrationHazard[j][i] = TransMatrixResults[j][i] / TransDenLambdaResults[i]
  #Default Category
  RatingMgrationHazard[RatingMgrationHazard.index=="Default"] = 0.0 # All the others = 0
  RatingMgrationHazard["Default"]["Default"] = 1.0 # Absorbinb

  RatingMgrationHazard = pd.concat([RatingMgrationHazard,
                                    pd.DataFrame({f"{rating}":0 for rating in final_states},
                                                 index=["NR"])])

  #1
  lmax = 0
  for i in initial_states:
      if (np.abs(RatingMgrationHazard[i][i]) > lmax):
          lmax = np.abs(RatingMgrationHazard[i][i])

  #2
  mat1 = np.zeros((22, 22))
  np.fill_diagonal(mat1, lmax)


  Lstar = RatingMgrationHazard +  mat1

  tmp = la.expm(Lstar)

  vec1 = np.zeros((22, 22))

  np.fill_diagonal(vec1, np.exp(-lmax))

  mexpgenerator = np.dot(vec1, tmp)

  mexpgeneratorDF = pd.DataFrame(mexpgenerator)

  # Change the column names
  mexpgeneratorDF.columns =["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
                      "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
                      "CCC+", "CCC", "CCC-", "CC", 'Default', 'NR']

  # Change the row indexes
  mexpgeneratorDF.index = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
                      "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
                      "CCC+", "CCC", "CCC-", "CC", 'Default', 'NR']

  # evals, evecs = la.eig(RatingMgrationHazard.values)

  # size = len(evals)
  # arr1 = np.zeros((size, size))
  # np.fill_diagonal(arr1, np.real(evals))

  # Recunstructed = np.dot(np.dot(evecs,arr1),np.linalg.inv(evecs))

  # Recunstructed = pd.DataFrame(Recunstructed)
  # Recunstructed.columns =["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
  #                     "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
  #                     "CCC+", "CCC", "CCC-", "CC",  'Default', 'NR']
  # Recunstructed.index = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
  #                     "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
  #                     "CCC+", "CCC", "CCC-", "CC",  'Default', 'NR']


  # arr1 = np.zeros((size, size))
  # np.fill_diagonal(arr1, np.real(np.exp(evals)))
  # Recunstructed = np.dot(np.dot(evecs,arr1),np.linalg.inv(evecs))
  # Recunstructed = pd.DataFrame(Recunstructed)
  # Recunstructed.columns =["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
  #                     "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
  #                     "CCC+", "CCC", "CCC-", "CC", 'Default', 'NR']
  # Recunstructed.index = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
  #                     "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
  #                     "CCC+", "CCC", "CCC-", "CC",  'Default', 'NR']


  return mexpgeneratorDF["Default"]

def compute_nonparametric_bootstrap_pd(ratings, year, Nt, B=10000):

  ratings = create_year_data(ratings, year)

  pds = np.zeros(shape=(22, B))

  for i in tqdm(range(B)):
    # print(f"\n{i}\n")

    rating_sample = ratings.sample(n=int(Nt[str(year[0])].sum()), replace=True)

    rating_sample = create_year_data(rating_sample, year)

    result = postprocess_transition_results(compute_nijk_v2(rating_sample, year))

    pds[:, i] = compute_default_probas(result)

  return pds

In [17]:
# results = compute_nijk(TransitionData, [2010])

In [18]:
## lambda 2010
# results[-1]

In [19]:
## N_R 2010
# results[1]

In [20]:
## nij 2010
# results[0]

In [21]:
# yearly_results = {f"{year}": compute_nijk(TransitionData, [year])\
#                   for year in sorted(TransitionData.Date.dt.year.unique())}

In [22]:
# yearly_results_postprocessed = {f"{year}": postprocess_transition_results(result)
#                   for year, result in yearly_results.items()}

In [23]:
initial_states = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
                     "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
                     "CCC+", "CCC", "CCC-", "CC", "Default"]

final_states = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+",
                    "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-",
                    "CCC+", "CCC", "CCC-", "CC", "Default", "NR"]

In [24]:
# compute_default_probas(postprocess_transition_results(compute_nijk(TransitionData, [2010])))

In [25]:
# compute_default_probas(yearly_results_postprocessed["2010"])

In [26]:
# Nts = pd.concat([yearly_results_postprocessed[str(year)][1]\
#            for year in sorted(TransitionData.Date.dt.year.unique())],
#           axis=1)
# Nts.columns = sorted(TransitionData.Date.dt.year.unique())
# Nts

In [27]:
# Nts.to_csv("Nts.csv", index=False)
# TransitionData.to_csv("TransitionData.csv", index=False)

Nts = pd.read_csv("Nts.csv")
TransitionData = pd.read_csv("TransitionData.csv")

In [28]:
TransitionData.Date = pd.to_datetime(TransitionData.Date)

In [29]:
yr = 2019
B = 1000

In [ ]:
pds = compute_nonparametric_bootstrap_pd(TransitionData, [yr], Nts, B=B)
pds

 68%|██████▊   | 682/1000 [6:29:48<2:54:53, 33.00s/it]

In [ ]:
pd.DataFrame(pds).dropna(axis=1).mean(axis=1)

In [ ]:
pd.DataFrame(pds).dropna(axis=1).quantile(axis=1, q=[0.025, 0.975])*10000

In [ ]:
pd.DataFrame(pds).to_csv(f"pds_{yr}_{B}.csv", index=False)

#### Parametric

In [ ]:
def compute_parametric_bootstrap_ci(pd, nr, alpha=0.05):
  pass

#### Non parametric